In [ ]:
from datetime import datetime
print(f"Execution Start Time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")


In [ ]:
# Package upgrade skipped - using environment default
print('Using default sagemaker_studio package from environment')


# Test 04: GDC Read/Write using Athena

This notebook tests reading and writing to the Glue Data Catalog using Athena

In [0]:
from sagemaker_studio import sqlutils

In [0]:
sql_output_4phn = sqlutils.sql("-- Comprehensive weather trends analysis combining monthly patterns, seasonal variations, and extreme weather events\nWITH monthly_trends AS (\n    SELECT \n        \"year\",\n        \"month\",\n        \"season\",\n        COUNT(*) as \"total_days\",\n        ROUND(AVG(\"temperature_celsius\"), 2) as \"avg_temperature\",\n        ROUND(AVG(\"humidity_percent\"), 2) as \"avg_humidity\",\n        ROUND(AVG(\"pressure_hpa\"), 2) as \"avg_pressure\",\n        ROUND(AVG(\"wind_speed_kmh\"), 2) as \"avg_wind_speed\",\n        ROUND(SUM(\"precipitation_mm\"), 2) as \"total_precipitation\",\n        ROUND(AVG(\"uv_index\"), 2) as \"avg_uv_index\",\n        ROUND(AVG(\"visibility_km\"), 2) as \"avg_visibility\"\n    FROM \"thunder_awsdatacatalog_test\".\"weather\"\n    GROUP BY \"year\", \"month\", \"season\"\n),\nseasonal_comparison AS (\n    SELECT \n        \"season\",\n        \"year\",\n        ROUND(AVG(\"temperature_celsius\"), 2) as \"seasonal_avg_temp\",\n        ROUND(MAX(\"temperature_celsius\"), 2) as \"seasonal_max_temp\",\n        ROUND(MIN(\"temperature_celsius\"), 2) as \"seasonal_min_temp\",\n        ROUND(SUM(\"precipitation_mm\"), 2) as \"seasonal_precipitation\",\n        COUNT(CASE WHEN \"temperature_celsius\" > 30 THEN 1 END) as \"hot_days\",\n        COUNT(CASE WHEN \"temperature_celsius\" < 0 THEN 1 END) as \"freezing_days\"\n    FROM \"thunder_awsdatacatalog_test\".\"weather\"\n    GROUP BY \"season\", \"year\"\n)\nSELECT \n    mt.\"year\",\n    mt.\"month\",\n    mt.\"season\",\n    mt.\"total_days\",\n    mt.\"avg_temperature\",\n    mt.\"avg_humidity\",\n    mt.\"avg_pressure\",\n    mt.\"avg_wind_speed\",\n    mt.\"total_precipitation\",\n    mt.\"avg_uv_index\",\n    mt.\"avg_visibility\",\n    sc.\"seasonal_avg_temp\",\n    sc.\"seasonal_max_temp\",\n    sc.\"seasonal_min_temp\",\n    sc.\"seasonal_precipitation\",\n    sc.\"hot_days\",\n    sc.\"freezing_days\",\n    -- Temperature deviation from seasonal average\n    ROUND(mt.\"avg_temperature\" - sc.\"seasonal_avg_temp\", 2) as \"temp_deviation_from_seasonal\",\n    -- Precipitation ranking within the year\n    RANK() OVER (PARTITION BY mt.\"year\" ORDER BY mt.\"total_precipitation\" DESC) as \"precip_rank_in_year\"\nFROM monthly_trends mt\nJOIN seasonal_comparison sc ON mt.\"season\" = sc.\"season\" AND mt.\"year\" = sc.\"year\"\nORDER BY mt.\"year\" DESC, mt.\"month\" ASC;", connection_id="bnecmtxcaqkgb4")
sql_output_4phn

,year,month,season,total_days,avg_temperature,avg_humidity,avg_pressure,avg_wind_speed,total_precipitation,avg_uv_index,avg_visibility,seasonal_avg_temp,seasonal_max_temp,seasonal_min_temp,seasonal_precipitation,hot_days,freezing_days,temp_deviation_from_seasonal,precip_rank_in_year
0,2023,1,Winter,31,7.90,83.34,1014.18,21.07,0.00,2.91,5.59,9.76,18.8,1.9,0.00,0,0,-1.86,1
1,2023,2,Winter,28,14.20,81.40,1016.79,22.66,0.00,4.10,7.34,9.76,18.8,1.9,0.00,0,0,4.44,1
2,2023,3,Spring,31,20.42,74.74,1017.54,21.19,0.00,5.36,7.75,27.76,43.4,11.5,0.00,38,0,-7.34,1
3,2023,4,Spring,30,28.35,62.97,1015.02,20.41,0.00,6.74,8.45,27.76,43.4,11.5,0.00,38,0,0.59,1
4,2023,5,Spring,31,34.53,51.49,1015.73,18.75,0.00,6.52,10.18,27.76,43.4,11.5,0.00,38,0,6.77,1
5,2023,6,Summer,30,37.96,41.33,1015.60,17.66,0.00,5.09,10.88,34.64,43.1,23.3,0.00,79,0,3.32,1
6,2023,7,Summer,31,35.23,36.70,1011.54,14.75,0.00,2.74,11.78,34.64,43.1,23.3,0.00,79,0,0.59,1
7,2023,8,Summer,31,30.85,36.78,1007.27,16.18,0.00,1.48,10.99,34.64,43.1,23.3,0.00,79,0,-3.79,1
8,2023,9,Autumn,30,22.53,45.91,1007.74,15.56,0.00,0.56,10.54,16.33,27.9,-0.4,0.00,0,1,6.20,1
9,2023,10,Autumn,31,16.42,56.18,1008.75,17.51,0.00,0.35,9.58,16.33,27.9,-0.4,0.00,0,1,0.09,1
